# Understanding SB 50 in LA, Part 1

For this project, we are going to do our best to recreate the analysis performed by Urban Footprint and summarized in [this blog post](https://urbanfootprint.com/analyzing-sb-50/).

Before getting started, make sure that you have the following packages installed: `pandas`, `geopandas`, `numpy`

In [1]:
# TODO: Import pandas, geopandas, numpy
import pandas as pd
import numpy as np
import geopandas as gpd


### Pre-Exercise
During the ITS Data Camp Course, we introduced you to the `folium` Python package to display interactive maps right in your browser, made possible because the folium package builds off the leaflet javascript package. For this exercise, we are going to continue to interactively explore spatial data, but we are going to use a different (but very similar) Python package called `ipyleaflet`, which has some additional functionality and plays a little bit better with GeoPandas DataFrames. Take a moment to read more about `ipyleaflet` [here](https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a). You will notice that the article mentions `folium` toward the end in a comparison of similar packages.

##### Install & Setup of ipyleaflet
Follow the instructions (using the conda install) on the [documentation page](https://ipyleaflet.readthedocs.io/en/latest/installation.html#jupyterlab-extension). If you are having difficulty getting the map to appear in your notebook (with everything else loading correctly), take a look at this [SO question](https://gis.stackexchange.com/questions/303261/ipyleaflet-map-object-doesnt-display-in-jupyter-notebook-but-it-gets-created) for the command to enable the extension. You will run this command not in the notebook, but instead in the command prompt (OSX) / anaconda prompt (Windows).

##### Test the Install
We can confirm that the package was installed properly and the extension was enabled by running the following code, which should display a map with a single point in the middle.

*Didn't work at first, but restarting my jupyter notebook server showed it without having to run the command from SO –Eric* 

In [2]:
from ipyleaflet import Map, Marker

center=(34.052235,-118.243683)
m = Map(center=center, zoom=10)
marker = Marker(location=center, draggable=True)
m.add_layer(marker);

m

Map(center=[34.052235, -118.243683], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

## Transportation-Based Criteria
Let's focus exclusively on service provided by LA Metro for now. There are two different ways that high-quality transit areas were defined in the legislation:
1. 1/2 mile from any major rail stop
2. 1/4 mile of any high-quality bus stop

### Major Rail Stops

##### Step 1: Import the Metro Rail Stops
Let's start by getting the Metro Rail stops from the [LA City GeoHub](http://geohub.lacity.org/datasets/6679d1ccc3744a7f87f7855e7ce33395_1?geometry=-119.297%2C33.719%2C-117.230%2C34.118). Although we could simply download the data and import it into our folder, let's practice calling the API to get the data. Once you make a successful call, save it to our `data/raw` directory.

In [3]:
# TODO: Import requests & json packages
import requests
import json
# TODO: Call the API to get the GeoJSON data, and save to 'data/raw'
gjson = requests.get('https://opendata.arcgis.com/datasets/6679d1ccc3744a7f87f7855e7ce33395_1.geojson').json()
#!mkdir data/raw
with open('data/raw/metrorail.gjson', 'w') as outfile:
    json.dump(gjson, outfile)

##### Step 2: Convert to GeoPandas DataFrame
Now that we have our data saved to disk, let's go ahead and read it back in and convert it to a GeoPandas dataframe. Because we are going to perform spatial manipulation, we need to make sure we first explicitly state our CRS and then convert it to an appropriate CRS for perform spatial manipulations (I recommend using NAD-83). Take a look at the GeoPandas documentation [here](http://geopandas.org/projections.html) for setting projections and re-projecting data.

_For users experiencing an issue with projections: It is likely that somehow your conda installation provided you an older version of pyproj (which you can check). You may need to remove `pyproj` via `conda remove --force pyrpoj` and then reinstall it using pip to get pyproj 2.4.x_ 

In [4]:
# TODO: Read in to GeoPandas Dataframe
rail_stops_gdf = gpd.read_file('data/raw/metrorail.gjson')

In [5]:
# TODO: Examine the DF head to make sure it read in correctly
rail_stops_gdf.head()

,OBJECTID,MetroLine,Station,StopNumber,TOOLTIP,NLA_URL,geometry
0,1,Blue Line,Downtown Long Beach Station,80101,Stop: Downtown Long Beach Station\nStop No: 80...,http://www.metro.net/riding/maps/blue-line/?nl...,POINT (-118.19292 33.76807)
1,2,Blue Line,Pacific Ave Station,80102,Stop: Pacific Ave Station\nStop No: 80102\nBlu...,http://www.metro.net/riding/maps/blue-line/?nl...,POINT (-118.19370 33.77226)
2,3,Blue Line,Anaheim Street Station,80105,Stop: Anaheim Street Station\nStop No: 80105\n...,http://www.metro.net/riding/maps/blue-line/?nl...,POINT (-118.18938 33.78183)
3,4,Blue Line,Pacific Coast Hwy Station,80106,Stop: Pacific Coast Hwy Station\nStop No: 8010...,http://www.metro.net/riding/maps/blue-line/?nl...,POINT (-118.18938 33.78909)
4,5,Blue Line,Willow Street Station,80107,Stop: Willow Street Station\nStop No: 80107\nB...,http://www.metro.net/riding/maps/blue-line/?nl...,POINT (-118.18983 33.80708)


In [6]:
# TODO: Examine current CRS
rail_stops_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
# TODO: Re-Project to NAD83
#went with this one https://spatialreference.org/ref/epsg/2229/ 
rail_stops_nad83 = rail_stops_gdf.to_crs('EPSG:2229')

In [8]:
rail_stops_nad83.crs

<Projected CRS: EPSG:2229>
Name: NAD83 / California zone 5 (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: USA - California - SPCS83 - 5
- bounds: (-121.42, 32.76, -114.12, 35.81)
Coordinate Operation:
- name: SPCS83 California zone 5 (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

##### Step 3: Buffer 1/2 mile
Now that we've re-projected our data, we can go ahead and perform spatial manipulations. Let's create a 1/2 mile buffer around each of the rail stops. _(Pro tip: Each CRS has it's own unit of measurement. Make sure you keep track of what you are currently using!)_ Once we do this, we are going to project it back to WGS84 so that we display it in our map below. We will also want to make sure we save to disk at `data/processed` so that we have it if needed later. 

In [9]:
# TODO: buffer 1/2 mi
# You could have used either ft or m, as long as it aligns with your CRS
rail_stops_halfmi_buff = rail_stops_nad83['geometry'].buffer(2640)

# TODO: Add original rail data to buffer
rail_stops_nad83.geometry = rail_stops_halfmi_buff

# TODO: Re-project back to WGS84 for map display
rail_stop_buff_wgs84 = rail_stops_nad83.to_crs('EPSG:4326')

# TODO: Write out to data/processed
!mkdir data/processed
rail_stop_buff_wgs84.to_file('data/processed/metro_rail_stops_buffered.geojson', driver='GeoJSON')

mkdir: data/processed: File exists


*I was very confused as to why my buffer was ~3x what it should be, until I realized I must have ran the cell multiple times and the way it's set up the buffer will increment each time! Perhaps most people won't run into this issue, but could structure the cell to be more robust, as below. -Eric*

In [10]:
# TODO: buffer 1/2 mi
# You could have used either ft or m, as long as it aligns with your CRS

# Basing the buffer on a freshly reprojected geometry is one option to avoid the circular reference/unwanted increment
rail_stops_halfmi_buff = rail_stops_gdf.to_crs('EPSG:2229')['geometry'].buffer(2640)

# TODO: Add original rail data to buffer
rail_stops_nad83.geometry = rail_stops_halfmi_buff

# TODO: Re-project back to WGS84 for map display
rail_stop_buff_wgs84 = rail_stops_nad83.to_crs('EPSG:4326')

# TODO: Write out to data/processed
!mkdir data/processed
rail_stop_buff_wgs84.to_file('data/processed/metro_rail_stops_buffered.geojson', driver='GeoJSON')

mkdir: data/processed: File exists


In [11]:
# View the head of the table and confirm that the geometry is showing POLYGON instead of POINT
rail_stop_buff_wgs84.head()

,OBJECTID,MetroLine,Station,StopNumber,TOOLTIP,NLA_URL,geometry
0,1,Blue Line,Downtown Long Beach Station,80101,Stop: Downtown Long Beach Station\nStop No: 80...,http://www.metro.net/riding/maps/blue-line/?nl...,"POLYGON ((-118.18424 33.76808, -118.18428 33.7..."
1,2,Blue Line,Pacific Ave Station,80102,Stop: Pacific Ave Station\nStop No: 80102\nBlu...,http://www.metro.net/riding/maps/blue-line/?nl...,"POLYGON ((-118.18501 33.77227, -118.18505 33.7..."
2,3,Blue Line,Anaheim Street Station,80105,Stop: Anaheim Street Station\nStop No: 80105\n...,http://www.metro.net/riding/maps/blue-line/?nl...,"POLYGON ((-118.18070 33.78184, -118.18074 33.7..."
3,4,Blue Line,Pacific Coast Hwy Station,80106,Stop: Pacific Coast Hwy Station\nStop No: 8010...,http://www.metro.net/riding/maps/blue-line/?nl...,"POLYGON ((-118.18069 33.78910, -118.18073 33.7..."
4,5,Blue Line,Willow Street Station,80107,Stop: Willow Street Station\nStop No: 80107\nB...,http://www.metro.net/riding/maps/blue-line/?nl...,"POLYGON ((-118.18114 33.80709, -118.18118 33.8..."


Let's confirm that the buffer looks correct by displaying it (along with the rail stops) on the map, following the example [here](https://ipyleaflet.readthedocs.io/en/latest/api_reference/geodata.html). You should see circle polygons that look to have a radius of about 1/2 mi.  

In [13]:
# TODO: Display on map
from ipyleaflet import Map, GeoData, basemaps, LayersControl

m = Map(center=(34, -118), zoom = 10, basemap= basemaps.Esri.WorldTopoMap)

geo_data = GeoData(geo_dataframe = rail_stop_buff_wgs84,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Rail Stops')

m.add_layer(geo_data)
m.add_control(LayersControl())

m

Map(center=[34, -118], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

Nice work! Let's move on to high-quality bus corridors.

### High Quality Bus Corridors
In addition to looking at neighborhoods surrounding rail stops, the legislation also allows for additional density near "high-quality bus stops". We will use Urban Footprint's definition of High Quality Bus Corridor as having the following criteria:
- less than a 10-minute average headway for three consecutive hours during the morning peak, from 6 to 10 AM, and the evening peak, from 3 to 7 PM.
- maintain an average headway below 20 minutes over the course of a weekday from 6 AM to 10 PM and an average headway below 30 minutes on weekends
  
Because this definition is a bit trickier, we are going to look a bit deeper through Metro's GTFS data. GTFS is a standard developed by Google showing scheduled _(not realtime)_ data. [TransitWiki](https://www.transitwiki.org/TransitWiki/index.php/General_Transit_Feed_Specification) and the [Google developers page](https://developers.google.com/transit/gtfs/) both provide a good overview of the GTFS standard; take some time to look through the documentation on each. Metro provides it's own GTFS data for easy consumption [here](http://developer.metro.net/gtfs-schedule-data/), with a README containing information on each archive.

##### Working with GTFS Data
There have been numerous development efforts (in multiple programming languages) that make it easier to ingest, analyze, view, and produce GTFS data, many of which you can see [here](https://github.com/CUTR-at-USF/awesome-transit). We will [use] one of the [GTFS libraries](https://github.com/CUTR-at-USF/awesome-transit#gtfs-libraries) to help with the ingestion of our data, but the rest we will do using `GeoPandas`. For this exercise we will be specifically focused on calculating frequency, but feel free to also explore the myriad of tools that have been developed to look at GTFS with another objective in mind.

##### Our Plan
Transit agencies _may_ optionally publish a table called `frequencies.txt`, which makes it fairly easy to calculate the average frequency at the peak hours. However, you will notice that Metro's archive doesn't contain that file, which means we will need to calculate it ourselves.

While looking through [Metro's bus GTFS data](http://developer.metro.net/gtfs-schedule-data/), we decide we are interested in 3 tables: 
- `trips`: This specifies which routes are bus routes. This is our starting point.
- `stop_times`: Combined with trips, we will calculate the arrival frequency using the criteria above to determine the High Quality Routes
- `stops`: Once we determine the High Quality Bus Routes, we will want to create the 1/4 mile buffer around the stops

##### Step 1: Install `partridge` package
We will use the `partridge` package for lightweight parsing of GTFS files (to keep memory overhead low) into `GeoPandas` dataframes that we can then work on for further analysis. Follow the install instructions [here](https://github.com/remix/partridge).

##### Step 2: Download Data, Get busiest day
Begin by downloading the entire LA Metro Bus GTFS zipped file to `data/raw/gtfs_bus-master.zip`. You will notice that the entire size of the folder is around 40MB zipped (and 237MB unzipped!). Whlie data of that size is not an issue storing on disk, loading all of it into [memory (RAM)](https://www.backblaze.com/blog/whats-diff-ram-vs-storage/) will be wasteful and unnecessary. We won't need to unzip the file as `partridge` is designed to directly read from zipped directories.

We are going to use `partridge` to get the service IDs that are associated with the busiest day in the GTFS feed.

In [16]:
# Import partridge package
import partridge as ptg

# TODO: Set inpath to zipped directory
inpath = 'data/raw/gtfs_bus-master.zip'

# TODO: follow partridge package quickstart example 
#       to get date, service_ids for the busiest day
date, service_ids = ptg.read_busiest_date(inpath)
#Specify view using service_ids from busiest day
view = {'trips.txt': {'service_id': service_ids}}
#Load GTFS feed filtered to service IDs that are associated with the busiest day
feed = ptg.load_feed(inpath, view)

#### Could use more detailed instructions here, maybe specifically refer to 'Extracting a new feed' section.
Also some may not get that the 'quickstart example' simply refers to the same readme as the install instructions -Eric

Let's see what's in this (now filtered) feed.

*This probably won't be intuitive to people without at least a basic understanding of attributes -Eric*

In [17]:
# Print contents of the feed
print([i for i in dir(feed)])

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_bootstrap', '_cache', '_config', '_convert_types', '_delete_after_reading', '_filter', '_locks', '_pathmap', '_prune', '_read', '_read_csv', '_shared_lock', '_transform', '_view', 'agency', 'calendar', 'calendar_dates', 'fare_attributes', 'fare_rules', 'feed_info', 'frequencies', 'get', 'routes', 'set', 'shapes', 'stop_times', 'stops', 'transfers', 'trips']


In [18]:
feed.agency

,agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone
0,LACMTA,Metro - Los Angeles,https://www.metro.net,America/Los_Angeles,en,(323)466-3876


In [16]:
# TODO: Pull stop times from the feed and examine the head of the table
stop_times_df = feed.stop_times

# Examine the head of the table
stop_times_df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type
0,49404780-DEC19-D05CAR-1_Weekday,76200.0,76200.0,30006,1,102 - Huntington Park - Palm - Seville,0,0
1,49404780-DEC19-D05CAR-1_Weekday,76260.0,76260.0,7437,2,102 - Huntington Park - Palm - Seville,0,0
2,49404780-DEC19-D05CAR-1_Weekday,76320.0,76320.0,6134,3,102 - Huntington Park - Palm - Seville,0,0
3,49404780-DEC19-D05CAR-1_Weekday,76380.0,76380.0,36554,4,102 - Huntington Park - Palm - Seville,0,0
4,49404780-DEC19-D05CAR-1_Weekday,76440.0,76440.0,36555,5,102 - Huntington Park - Palm - Seville,0,0


In [17]:
# TODO: Extract the trips and examine the head of the table
trips_df = feed.trips
trips_df.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,102-13131,DEC19-D05CAR-1_Weekday,49404780-DEC19-D05CAR-1_Weekday,NaN,0,1020400,1020065_DEC19
1,102-13131,DEC19-D05CAR-1_Weekday,49404781-DEC19-D05CAR-1_Weekday,NaN,0,1020100,1020037_DEC19
2,102-13131,DEC19-D05CAR-1_Weekday,49404782-DEC19-D05CAR-1_Weekday,NaN,0,1020300,1020038_DEC19
3,102-13131,DEC19-D05CAR-1_Weekday,49404783-DEC19-D05CAR-1_Weekday,NaN,0,1020300,1020037_DEC19
4,102-13131,DEC19-D05CAR-1_Weekday,49404784-DEC19-D05CAR-1_Weekday,NaN,1,1020300,1020061_DEC19


##### Step 3: Write helper function

Write a function that filters the stop_times for only the hours of interest. You'll notice that the times are in seconds, so make sure to account for the appropriate conversion. We will do our calculations based on the `arrival_time`.

In [18]:
# TODO: Create a function to filter our stop_times dataframe
#       by a beginning hour (inclusive) and end hour (exclusive)
#       using methods in the Pandas library
def filter_stoptimes_byhour(df, begin_hour, end_hour):
    ''' 
    Filter a GTFS DF by start/end hour
    
    :param df DataFrame: Pandas DF of stop_times from partridge
    :param begin_hour int: begin hour (inclusive), 0-23
    :param end_hour int: end hour (not inclusive), 1-24
    returns: Filtered stop_times df
    '''
    begin_sec = begin_hour * 60**2
    end_sec = end_hour * 60**2
    
    df = df[df['arrival_time'] >= begin_sec]
    df = df[df['arrival_time'] < end_sec]
    return df

# Compare the length of raw & filtered dataframes for comparison
print(f'The unfiltered DF contains {len(stop_times_df)} rows.')
print(f'Between 7-10am, there are {len(filter_stoptimes_byhour(stop_times_df, 7, 10))} rows.')

The unfiltered DF contains 839248 rows.
Between 7-10am, there are 164903 rows.


### "The appropriate conversion" is pretty hard!
*Per [spec](https://developers.google.com/transit/gtfs/reference#stop_timestxt), GTFS arrival times can wrap 'above 24 hours' if the schedule continues overnight into the next day. Below cells show one option for correcting and the difference it makes in the early am hours. Of course it doesn't seem to matter in either peak period, but if we care about being precise we might want to point this out in the directions. -Eric*

In [19]:
print('seconds in a day: {}'.format(24*60**2))
print('max time in df: {}'.format(stop_times_df['arrival_time'].max()))

seconds in a day: 86400
max time in df: 107100.0


In [20]:
print('So the simple function won\'t capture all trips before {} hours, or 5:45AM'.format((107100-86400) / 60**2))

So the simple function won't capture all trips before 5.75 hours, or 5:45AM


In [21]:
def better_filter_stoptimes_byhour(df, begin_hour, end_hour):
    ''' 
    Filter a GTFS DF by start/end hour
    
    :param df DataFrame: Pandas DF of stop_times from partridge
    :param begin_hour int: begin hour (inclusive), 0-23
    :param end_hour int: end hour (not inclusive), 1-24
    returns: Filtered stop_times df
    '''
    sec_in_day = 24*60**2
    df['real_arr_time'] = df['arrival_time'] % sec_in_day
    
    begin_sec = begin_hour * 60**2
    end_sec = end_hour * 60**2
    
    df = df[df['real_arr_time'] >= begin_sec]
    df = df[df['real_arr_time'] < end_sec]
    df.drop(columns=['real_arr_time'], inplace=True)
    return df

# Compare the length of raw & filtered dataframes for comparison
print(f'The unfiltered DF contains {len(stop_times_df)} rows.')
#original function undercounts early AM
print(f'Between 3-5am, there are {len(filter_stoptimes_byhour(stop_times_df, 3, 5))} rows.')
#more advanced version counts it correctly...
print(f'Well, actually {len(better_filter_stoptimes_byhour(stop_times_df, 3, 5))} rows.')

The unfiltered DF contains 839248 rows.
Between 3-5am, there are 8488 rows.
Well, actually 14718 rows.


##### Step 4: Calculate Stop Average Frequency at Each Peak
As a reminder, The first criterion for High-Quality Bus Corridors is that the routes "have less than a 10-minute average headway for three consecutive hours" in the AM peak. During this step, we are going to filter for all the routes that have more than 18 scheduled arrivals in the 3 hour period between 7-10am.

With that helper function complete, we are ready to write the aggregation pipeline to compute the average frequency for each hour in the `stop_times` dataframe. _This next step will involve a fairly heavy use of the `pandas` library. I recommend [brushing up](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html) on some of the most common operations if the steps below seem intimidating. Also, note that the `pandas` library often has multiple methods that essentially perform the same function, so how you complete the next step will look a little different depending on the methods you choose._ 
  
We will calculate the frequency through the following steps:
1. Filter for hours by using our new function `filter_stoptimes_byhour`.
2. Join `stop_times` to `trips`, joining on the `trip_id`.
3. Group by the following:
    - `route_id`
    - `direction_id`
    - `stop_id`
    - `service_id`
4. Summarize by count into a new column called `count`
5. Filter for rows that have `count` >= 18.
6. Create a new column `am_headway` that shows the average headway (in minutes).

*Looking at your completed version, it seems like I mostly did ok except for failing to specify an 'inner' merge. Actually looked it up and seems like that's the default, so the discrepancy probably isn't due to that. Besides being less elegant, curious to know why mine gives slightly longer df's*

In [23]:
# TODO: Perform aggregation & computation steps from above
st_filtered_am = better_filter_stoptimes_byhour(stop_times_df, 7, 10)
stop_freq_am_df = trips_df.merge(st_filtered_am, on='trip_id')
stop_freq_am_df = stop_freq_am_df.groupby(['route_id', 'direction_id', 'stop_id', 'service_id']).count()
stop_freq_am_df = stop_freq_am_df.loc[stop_freq_am_df['trip_id'] >= 18]
stop_freq_am_df['am_headway'] = 3*60 / stop_freq_am_df['trip_id']
stop_freq_am_df = stop_freq_am_df.rename(columns={'trip_id':'count'})[['count', 'am_headway']]
stop_freq_am_df = stop_freq_am_df.reset_index()

# Exmaine the length and head of the new aggregated table
print(len(stop_freq_am_df))
stop_freq_am_df.head()

880


,route_id,direction_id,stop_id,service_id,count,am_headway
0,108-13131,0,13000148,DEC19-D05CAR-1_Weekday,18,10.000000
1,108-13131,0,1539,DEC19-D05CAR-1_Weekday,19,9.473684
2,108-13131,0,1637,DEC19-D05CAR-1_Weekday,20,9.000000
3,108-13131,0,1639,DEC19-D05CAR-1_Weekday,20,9.000000
4,108-13131,0,1642,DEC19-D05CAR-1_Weekday,20,9.000000


In addition to providing minimum of a 10 minute headway for the AM Peak, the stop must also meet the same minimum frequency for the PM Peak. Again, although the legislation gives a little bit of leeway with a 4 hour window, we are going to pick the hours of 4-7pm (the same hours used in the UF analysis). The new column should be named `pm_headway`.
  
_Hint: Make sure to use 24 hour format in your `filter_stoptimes_byhour` function_

In [31]:
# TODO: Perform same steps for PM Peak
st_filtered_pm = better_filter_stoptimes_byhour(stop_times_df, 16, 19)
stop_freq_pm_df = trips_df.merge(st_filtered_pm, on='trip_id')
stop_freq_pm_df = stop_freq_pm_df.groupby(['route_id', 'direction_id', 'stop_id', 'service_id']).count()
stop_freq_pm_df = stop_freq_pm_df.loc[stop_freq_pm_df['trip_id'] >= 18]
stop_freq_pm_df['pm_headway'] = 3*60 / stop_freq_pm_df['trip_id']
stop_freq_pm_df = stop_freq_pm_df.rename(columns={'trip_id':'count'})[['count', 'pm_headway']]
stop_freq_pm_df = stop_freq_pm_df.reset_index()
# TODO: Exmaine the length and head of the new aggregated table
print(len(stop_freq_pm_df))
stop_freq_pm_df.head()

965


,route_id,direction_id,stop_id,service_id,count,pm_headway
0,108-13131,0,1641,DEC19-D05CAR-1_Weekday,18,10.0
1,108-13131,0,1644,DEC19-D05CAR-1_Weekday,18,10.0
2,108-13131,0,1646,DEC19-D05CAR-1_Weekday,18,10.0
3,108-13131,0,1647,DEC19-D05CAR-1_Weekday,18,10.0
4,108-13131,0,1649,DEC19-D05CAR-1_Weekday,18,10.0


##### Step 5: Filter for those stops that meet both headway conditions
The stop must meet both AM and PM frequency criteria in order to be eligible for SB50 consideration. If you looked at the length of the two dataframes, you'll see that there is a difference in the number of rows, so we know that getting the intersection of both tables will produce (at most) the length of the smaller dataframe. 

To perform this filter, we need to do the following:
- Use a join to filter for rows with a `route_id` and `stop_id` that meet both criteria (`direction_id` can be different).
- Because a route has multiple directions, it is possible that a route would have frequent service in both directions at the same time, resulting in either of the tables having multiple rows with the same `route_id` and `stop_id`. Performing the join will result in the [cartesian product](https://www.geeksforgeeks.org/sql-join-cartesian-join-self-join/) of the matches. It shouldn't be too many, but make sure that we remove any rows with duplicate (`route_id`, `stop_id`) after the join.

In [33]:
# TODO: Find route_id, stop_id that meet both criteria
ampm_stop_freq_df = stop_freq_am_df.merge(stop_freq_pm_df, on=['route_id', 'stop_id'])
ampm_stop_freq_df = ampm_stop_freq_df.drop_duplicates(subset=['route_id', 'stop_id'])
# TODO: Print the length and head of the new table for confirmation
print(len(ampm_stop_freq_df))
ampm_stop_freq_df.head()

557


,route_id,direction_id_x,stop_id,service_id_x,count_x,am_headway,direction_id_y,service_id_y,count_y,pm_headway
0,108-13131,0,1646,DEC19-D05CAR-1_Weekday,20,9.000000,0,DEC19-D05CAR-1_Weekday,18,10.000000
1,108-13131,0,1647,DEC19-D05CAR-1_Weekday,21,8.571429,0,DEC19-D05CAR-1_Weekday,18,10.000000
2,108-13131,0,6275,DEC19-D05CAR-1_Weekday,22,8.181818,0,DEC19-D05CAR-1_Weekday,18,10.000000
3,108-13131,0,8679,DEC19-D05CAR-1_Weekday,21,8.571429,0,DEC19-D05CAR-1_Weekday,18,10.000000
4,108-13131,1,16262,DEC19-D05CAR-1_Weekday,18,10.000000,1,DEC19-D05CAR-1_Weekday,21,8.571429


Now that we have the high-frequency stops, we can join it to  the `stops` feed to get the geometry objects for the buffer. Let's keep only the `stop_id`, `stop_name`, `am_headway`, `pm_headway`, `stop_lat`, `stop_lon` columns, dropping the rest.

In [34]:
# TODO: Pull stops from the feed and examine the head of the table
stops_df = feed.stops
display(stops_df.head())

# TODO: Join stops with stop_freq_df and drop unnecessary columns
stops_joined_df = pd.merge(stops_df, ampm_stop_freq_df, on='stop_id')
stops_joined_df = stops_joined_df[['stop_id', 'stop_name', 'am_headway', 'pm_headway',
                                 'stop_lat', 'stop_lon']]

# TODO: Exmaine the head of the final df
stops_joined_df.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,stop_url,location_type,parent_station,tpis_name
0,1,1,Paramount / Slauson,NaN,33.973248,-118.113113,NaN,NaN,NaN,NaN
1,3,3,Jefferson / 10th,NaN,34.025471,-118.328402,NaN,NaN,NaN,NaN
2,6,6,120th / Augustus F Hawkins,NaN,33.924696,-118.242222,NaN,NaN,NaN,NaN
3,7,7,120th / Martin Luther King Hospital,NaN,33.924505,-118.240369,NaN,NaN,NaN,NaN
4,8,8,120th / Crenshaw,NaN,33.923818,-118.326886,NaN,NaN,NaN,NaN


,stop_id,stop_name,am_headway,pm_headway,stop_lat,stop_lon
0,46,7th / Hoover,6.923077,6.923077,34.059621,-118.283965
1,70,El Monte Busway / Alameda - Union Station,5.806452,6.428571,34.054490,-118.237358
2,99,Olive / Olympic,7.500000,9.000000,34.042134,-118.259555
3,213,Avalon / Martin Luther King Jr,6.206897,6.206897,34.010870,-118.265404
4,215,Avalon / Vernon,6.206897,6.206897,34.004085,-118.265376


##### Step 7: Create 1/4 mi. buffers around High-Quality Transit Corridor Stops
After the join, follow the same general steps as with the rail stations to create buffers around each of these stops (but this time for 1/4 mi. instead of 1/2 mi.).



*Might also remind to assign a CRS (and why WGS84 is appropriate) before transforming to NAD83, since unlike the rail station data there is no preset CRS when converting a gdf from xy coordinates in a df. –Eric*

In [38]:
# TODO: Convert to gdf, project, buffer
bus_gdf = gpd.GeoDataFrame(
    stops_joined_df, geometry=gpd.points_from_xy(stops_joined_df['stop_lon'],
                                                       stops_joined_df['stop_lat']))
bus_gdf.crs = 'EPSG:4326'
bus_gdf_nad83 = bus_gdf.to_crs('EPSG:2229')

# You could have used either ft or m, as long as it aligns with your CRS
bus_qtrmi_buff = bus_gdf_nad83['geometry'].buffer(1320)

# TODO: Add original rail data to buffer
bus_gdf.geometry = bus_qtrmi_buff

# TODO: Re-project back to WGS84 for map display
bus_buff_wgs84 = bus_gdf.to_crs('EPSG:4326')

Go ahead and display the buffers on a map using `ipyleaflet`.

In [40]:
# TODO: Create map object, add buffers, and display
from ipyleaflet import Map, GeoData, basemaps, LayersControl

m = Map(center=(34, -118), zoom = 10, basemap= basemaps.Esri.WorldTopoMap)

geo_data = GeoData(geo_dataframe = bus_buff_wgs84,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Bus Stops')

m.add_layer(geo_data)
m.add_control(LayersControl())

m


Map(center=[34, -118], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

Finally, as with the rail stop, let's go ahead and save this buffered geometry to `data/processed/hqbus_stop_buff_wgs84.geojson`.

In [60]:
# TODO: Write out to data/processed
bus_buff_wgs84.to_file('data/processed/hqbus_stop_buff_wgs84.geojson', driver='GeoJSON')

#### Cool! 

*But I was surprised to see that most of Wilshire didn't show up on the map. Played around with things for a while to see if I messed up a join somewhere, if I did I couldn't figure out where.*

### To Be Continued...
In the next part, we will add the non-transportation criteria for areas that were considered for SB50. Stay tuned.